In [53]:
import numpy as np
import time
import random
from numba import njit


In [54]:
@njit()
def _calculate_cost(chromosome, flow_matrix, distance_matrix):
  cost = 0
  n = chromosome.shape[0]
  for i in range(n):
    for j in range(n):
      cost += distance_matrix[i][j]*flow_matrix[chromosome[i]][chromosome[j]]
  return cost

In [55]:
class Individual:

  def __init__(self, size,chromosome):
    self.chromosome = chromosome
    self.size = size
    self.cost = np.inf

  def mutation(self, mut_prob):
    if random.random() <= mut_prob:
      mp1 = random.randint(0, self.size - 1)
      mp2 = random.randint(0, self.size - 1)
      self.chromosome[mp1], self.chromosome[mp2] = self.chromosome[mp2], self.chromosome[mp1]

  @staticmethod
  def generateRandom(n):
    return Individual(n,np.random.permutation(n))

  def cutAndCrossfill(self,parent,cro_prob):

    def insertValues(child, parent, n, crossover_point):
      while(len(child.chromosome) != n):
        if(parent.chromosome[crossover_point] not in child.chromosome):
          child.chromosome = np.append(child.chromosome, parent.chromosome[crossover_point])
        crossover_point = (crossover_point + 1) % n

    if(random.random() > cro_prob):
      return Individual(self.size,self.chromosome), Individual(parent.size,parent.chromosome)
    
    n = self.size
    crossover_point = random.randint(0, n)

    child1 = Individual(n,[])
    child2 = Individual(n,[])
    child1.chromosome = self.chromosome[:crossover_point]
    child2.chromosome = parent.chromosome[:crossover_point]

    insertValues(child1, parent, n, crossover_point)
    insertValues(child2, self, n, crossover_point)

    return (child1, child2)

  def calculate_cost(self,flow_matrix,distance_matrix):  
    self.cost = _calculate_cost(self.chromosome, flow_matrix,distance_matrix)
    
  def __str__(self):
    return str(self.chromosome)

  def __lt__(self,other):
    return self.cost < other.cost

  def __gt__(self,other):
    return self.cost > other.cost

  def __eq__(self,other):
    return self.cost == other.cost


In [56]:
def bestTwoOutOfFive(population):
    # Select 5 individuals randomly
    tournament = [population[random.randint(
        0, len(population)-1)] for _ in range(len(population)//5)]

    # return best 2 of five
    return sorted(tournament)[:2]


def replaceWorst(population, newIndividual):
  population = sorted(population)
  if(newIndividual.cost < population[-1].cost):
    population[-1] = newIndividual


In [57]:
def geneticAgorithm(f, d, pop_size, cro_prob, mut_prob, ter_cond, bk_fitness):
    """
    Genetic Algori
    """
    # Variables Initialization
    
    generations = 1
    n = len(f[0])
    offspring = []

    # INITIALISE population with random candidate solutions
    population = [Individual.generateRandom(n) for _ in range(pop_size)]

    # EVALUATE each candidate    
    for ind in population:
        ind.calculate_cost(f,d)

    # Repeat until termination condition is satisfied
    while(generations < ter_cond + 1):
        # If optimal solution exists and it is reached
        if((bk_fitness is not None) and population[0].cost == bk_fitness):
            break

        
        for i in range(pop_size):

            # PARENT SELECTION (best two out of five)
            father, mother = bestTwoOutOfFive(population)

            # CROSSOVER
            child1, child2 = father.cutAndCrossfill(mother, cro_prob)

            # MUTATION
            child1.mutation(mut_prob)
            child2.mutation(mut_prob)

            # EVALUATION OF NEW CANDIDATES
            child1.calculate_cost(f,d)
            child2.calculate_cost(f,d)


            offspring.append(child1)
            offspring.append(child2)

        # SURVIVOR SELECTION
        population = sorted(offspring)[:pop_size]
        offspring = []

        generations += 1

    return population


In [58]:
def read_qap_dataset(filename):
    def read_integers(filename):
        with open(filename) as f:
            return [int(elem) for elem in f.read().split()]

    file_it = iter(read_integers(filename))
    # Number of points
    n = next(file_it)
    # Distance between locations
    A = [[next(file_it) for j in range(n)] for i in range(n)]
    # Flow between factories
    B = [[next(file_it) for j in range(n)] for i in range(n)]
    
    return (np.array(B), np.array(A))


In [59]:
def test(pop_size, cro_prob, mut_prob, ter_cod, runs, filename, best_cost=None):
    runs_stats = []

    f, d = read_qap_dataset(filename)

    for i in range(runs):
        print('Run number: {}'.format(i))
        begining = time.time()
        p = geneticAgorithm(f,d,pop_size,cro_prob,mut_prob,ter_cod,best_cost)
        end = time.time()

        total_time = end - begining
        print(f'Total time: {total_time}')
        best = p[0]
        worst = p[-1]
        average_cost = np.mean([ind.cost for ind in p])
        runs_stats.append({ 'runtime': total_time, 'best': best, 'worst': worst, 'average_cost': average_cost })
    
    costs = [ stat['average_cost'] for stat in runs_stats ]
    times = [ stat['runtime'] for stat in runs_stats ]
 
    best_individual = min([ stat['best'] for stat in runs_stats ])
    worst_individual = max([ stat['worst'] for stat in runs_stats ])

    cost_average = np.mean(costs)
    cost_std = np.std(costs)
    
    time_average = np.mean(times)
    time_std = np.std(times)
    

    max_time = max(times)
    min_time = min(times)

    print(f'Costo promedio: {cost_average}')
    print(f'Desviación estandar de costos: {cost_std}')
    print(f'Mejor solución encontrada: {best_individual}: {best_individual.cost}')
    print(f'Peor solución encontrada: {worst_individual}: {worst_individual.cost}')
    print(f'Tiempo promedio de ejecución: {time_average}')
    print(f'Desviación estandar tiempo de ejecución: {time_std}')
    print(f'Valor máximo (tiempo de ejecución): {max_time}')
    print(f'Valor mínimo (tiempo de ejecución): {min_time}')


## Parámetros 1
* Tamaño de población: 300
* Número máximo de generaciones: 50,000
* Probabilidad de cruzamiento: 0.9
* Probabilidad de mutación: 0.2
* Número de corridas: 30
* Dataset: lipa20a

In [60]:
POP_SIZE = 60
CRO_PROB = 0.9
MUT_PROB = 0.8
TER_COND = 1000
RUNS = 10
FILENAME = 'tho40.dat'

test(POP_SIZE,CRO_PROB,MUT_PROB,TER_COND,RUNS,FILENAME,best_cost=1)

Run number: 0
Total time: 15.689120292663574
Run number: 1


KeyboardInterrupt: 

**Recursos**

**Test cases**
https://neos-guide.org/content/quadratic-assignment-problem#:~:text=The%20quadratic%20assignment%20problem%20(QAP)%20was%20introduced%20by%20Koopmans%20and,minimize%20the%20total%20assignment%20cost

**Datasets**
http://www.mgi.polymtl.ca/anjos/qaplib/inst.html

**Todo**

**Prioridad Alta**
*   Esqueleto Clase (Dani) ✅✅
*   Esqueleto Algoritmo (Morfin) ✅✅
*   Mutación (Dani) ✅✅
*   Cruzamiento (Morfin) ✅✅
*   Selección de Padres (Dani) ✅✅
*   Reemplazo (Morfin) ✅✅
*   Fitness (Dani) ✅

**Prioridad Media**
*   Leer el archivo (Ambos) ✅✅
*   Calcular STD, AVG, max, min (Dani)✅
*   Guardar en archivo la mejor solución (Dani)

**Prioridad Baja**
*   Calibrar parámetros

**Necesario**
*   Redactar el reporte